# Exercício de Programação 5

<font color="red">**Prazo de submissão: 23:55 do dia 23.08.2021** </font>

2021.1 Álgebra Linear Computacional - DCC - UFMG

Erickson - Fabricio

Instruções:
* Antes de submeter suas soluções, certifique-se de que tudo roda como esperado. Primeiro, **reinicie o kernel** no menu, selecione Kernel$\rightarrow$Restart e então execute **todas as células** (no menu, Cell$\rightarrow$Run All)
* Apenas o arquivo .ipynb deve ser submetido. Ele não deve ser compactado.
* Não deixe de preencher seu nome e número de matrícula na célula a seguir

**Nome do aluno:** Daniel Oliveira Barbosa

**Matricula:** 2020006450

## Introdução
Esse EP será dividido em 2 partes:

1. Métodos iterativos para calcular autovalores e singular values.
3. Métodos iterativos para solução de sistemas $Ax = b$ (Jacob e Seidel)

Os conceitos cobrados aqui são discutidos nas aulas que abrangem o conteúdo de cada parte e elas devem ser a principal fonte de consulta para suas soluções.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

### Questão 1 - Cálculo de autovalores e valores singulares


Nessa primeira parte, vamos implementar o algoritmo QR para obtenção dos autovalores de uma matriz A, como foi descrito nas aulas.

Abaixo temos a matriz A:

In [3]:
# ATENÇÃO: Não editar esta célula
A = np.vander(np.array([5, 4, 3, 2, 1]))
print(A)

[[625 125  25   5   1]
 [256  64  16   4   1]
 [ 81  27   9   3   1]
 [ 16   8   4   2   1]
 [  1   1   1   1   1]]


**1.1.1** Implemente o algoritmo QR, utilizando um **while loop** até satisfazer a condição: $A_i[1][0] < 10^{-10}$. Para tanto, lembre-se dos passos do algoritmo discutidos em aula:

Inicializando:
$A_0 = A$ \\
Em cada loop: \\
$A_0 = Q_0R_0$ \\
$A_1 = Q_0^T A_0 Q_0$ \\
$A_0 \leftarrow A_1$ \\


Para a decomposição QR, utilize o comando [`np.linalg.qr`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.qr.html).

Ao final do loop imprima os resultados: \\

print(iter, A[1][0], A)

iter = número de iterações necessárias para convergência.

In [4]:
# Insira seu código aqui
A_0 = A
iter = 0
while(not(A_0[1][0] < 10**(-10))):
    Q_0, R_0 = np.linalg.qr(A_0)
    A_0 = Q_0.T @ A_0 @ Q_0
    iter += 1
A = A_0
print(iter, A[1][0], np.around(A, decimals=0))

8 5.732002112419937e-11 [[ 681. -130.   48.  -33.   17.]
 [   0.   18.   -4.    4.   -2.]
 [   0.    0.    2.   -0.    0.]
 [   0.    0.    0.    0.   -0.]
 [   0.    0.    0.    0.    0.]]


**1.1.2** Repita o exercício anterior, agora simplificando o algoritmo QR, considerando a relação:

$$Q_0^TA_0 = R_0$$

Ao final do loop imprima os resultados: \\

print(iter, R_0, A[1][0], A)

iter = número de iterações necessárias para convergência.

In [5]:
# Insira seu código aqui
A = np.vander(np.array([5, 4, 3, 2, 1]))
A_0 = A
iter = 0
while(not(A_0[1][0] < 10**(-10))):
    Q_0, R_0 = np.linalg.qr(A_0)
    A_0 = R_0 @ Q_0
    iter += 1
A = A_0
print(iter, R_0, A[1][0], np.around(A, decimals=0))

8 [[-6.80926658e+02  1.29785039e+02 -4.80047507e+01  3.29546793e+01
   1.68972529e+01]
 [ 0.00000000e+00 -1.76584868e+01  3.94000666e+00 -3.88787075e+00
  -2.17967102e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.97307233e+00  4.82572268e-01
   4.47583086e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.12353534e-01
  -1.89839670e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.94391956e-02]] 5.732002112419974e-11 [[ 681. -130.   48.  -33.   17.]
 [   0.   18.   -4.    4.   -2.]
 [   0.    0.    2.   -0.    0.]
 [   0.    0.    0.    0.   -0.]
 [   0.    0.    0.    0.    0.]]


O próximo passo agora é utilizar o algoritmo QR para encontrar valores singulares de matrizes não quadradas. Considere para este exercício as matrizes B e C abaixo:

In [6]:
B = np.array([[4, 5, 6], [7, 8, 9]])
C = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [10, 10, 7, 2]])
print('B:', '\n', B)
print('C:', '\n', C)

B: 
 [[4 5 6]
 [7 8 9]]
C: 
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [10 10  7  2]]


**1.2.1** Para resolver essa questão, lembre-se que para qualquer matriz não quadrada B:

1. Existe uma decomposição $B = U \Sigma V^T$ 

1. $B^T B$ é quadrada, simétrica e admite decomposição espectral da forma: $B^T B = Q \Lambda Q^T$

3. Substituindo 1 em 2, temos: $B^T B = V \Sigma^2 V^T$

A nível de algoritmo, crie uma função generalista que retorna os valores singulares de qualquer matriz real, não quadrada seguindo os passos:

1. Calcule $S =B^T B$
2. Use o algoritmo QR para obter autovalores de $S$ (loop com 10 iterações).
3. Tire a raiz quadrada dos autovalores de S para obter os valores singulares de B.
4. Retorne um vetor **apenas com os valores singulares não nulos** (dica: [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)).

Imprima a saída da sua função para B e C.

In [10]:
# Insira seu código aqui
def singValues(B):
    S = B.T @ B
    S0 = S
    iter = 0
    while(iter <= 10):
        Q, R = np.linalg.qr(S0)
        S0 = R @ Q
        iter += 1
    singValuesDiagonal = np.diag(S0) 
    singValuesVec = np.sqrt(singValuesDiagonal[singValuesDiagonal >= 0])
    for i in range(singValuesVec.size):
        if singValuesVec[i] < 10**(-5):
            singValuesVec = np.delete(singValuesVec, i)
    return singValuesVec
print("singB:", singValues(B))
print("singC:", singValues(C))

singB: [16.45601991  0.44655204]
singC: [20.34572265  6.55272059  0.33678326]


### Questão 2 -Gauss-Jacobi e Gauss Seidel

Nesta seçãoo, vamos implementar os algoritmos de Gauss-Jacobi e Gauss Seidel para resolver o sistema $Ax = b$, tal como foi descrito nas aulas.

Dica 1:

Passos iterativos do método Gauss-Jacobi para uma matriz 3x3:

Dado um valor inicial para x: $\vec{x}^{k = 0} = [1, 1, 1]$

Realize a iteração:

$x_1^{k+1} = (b_1 - a_{12} x_2^{(k)} - a_{13}x_3^{(k)})/a_{11}$

$x_2^{k+1} = (b_2 - a_{21} x_1^{(k)} - a_{23}x_3^{(k)})/a_{22}$

$x_3^{k+1} = (b_3 - a_{31} x_1^{(k)} - a_{32}x_2^{(k)})/a_{33}$

Dica 2:

Passos iterativos do método Gauss-Seidel para uma matriz 3x3:

Dado um valor inicial para x: $\vec{x}^{k = 0} = [1, 1, 1]$

Realize a iteração:

$x_1^{k+1} = (b_1 - a_{12} x_2^{(k)} - a_{13}x_3^{(k)})/a_{11}$

$x_2^{k+1} = (b_2 - a_{21} x_1^{(k+1)} - a_{23}x_3^{(k)})/a_{22}$

$x_3^{k+1} = (b_3 - a_{31} x_1^{(k+1)} - a_{32}x_2^{(k+1)})/a_{33}$

Dica 3:

Ao fazer o exercício no Colab tive problemas de arredondamento ao criar o vetor x_iter com o comando np.array. Caso tenham este problema, basta declarar x_iter como uma lista.

**2.1** Encontre a solução do sistema $Ax = b$, pelo método de Gauss-Jacobi realizando a iteração do método 5 vezes. A cada iteração, realize o print da sua estimativa para x. Neste exercício, considere a primeira estimativa de $x = [1, 1, 1]$, com A e b definidos abaixo.

Após rodar o algoritmo, faça um print da sua solução, comparando com a saída do método numpy: np.linalg.solve(A,b)

In [11]:
# ATENÇÃO: Não editar esta célula
A = np.array([[27, 6, -2], [6, 15, 2], [1, 1, 54]])
b = np.array([85, 72, 110])
print('A: \n', A)
print('b: \n', b)

A: 
 [[27  6 -2]
 [ 6 15  2]
 [ 1  1 54]]
b: 
 [ 85  72 110]


In [12]:
# Insira seu código aqui
x = [0, 0, 0]
x_iter = [0, 0, 0]
for i in range(5):
    x_iter[0] = (b[0] - A[0, 1] * x[1] - A[0, 2] * x[2]) * 1 / A[0, 0]
    x_iter[1] = (b[1] - A[1, 0] * x[0] - A[1, 2] * x[2]) * 1 / A[1, 1]
    x_iter[2] = (b[2] - A[2, 0] * x[0] - A[2, 1] * x[1]) * 1 / A[2, 2]
    x = x_iter
    print("iteration ", i+1, x)
    print()
f = np.linalg.solve(A, b)
print("np.linalg.solve: ", f)

iteration  1 [3.1481481481481484, 4.8, 2.037037037037037]

iteration  2 [2.2323731138545955, 3.635445816186557, 1.9283737235177565]

iteration  3 [2.4831137981450437, 3.549637984272948, 1.9253194114367038]

iteration  4 [2.5019559598976193, 3.5425083611827253, 1.9251025125725862]

iteration  5 [2.5035242540018086, 3.5419099633895987, 1.9250845515297887]

np.linalg.solve:  [2.50366797 3.54185509 1.92508291]


**2.2** Repita o exercício anterior, agora pelo método de Gauss-Seidel

In [14]:
# Insira seu código aqui
x = [0, 0, 0]
for i in range(5):
    x[0] = (b[0] - A[0, 1] * x[1] - A[0, 2] * x[2]) * 1 / A[0, 0]
    x[1] = (b[1] - A[1, 0] * x[0] - A[1, 2] * x[2]) * 1 / A[1, 1]
    x[2] = (b[2] - A[2, 0] * x[0] - A[2, 1] * x[1]) * 1 / A[2, 2]
    print("iteration ", i+1, x)
    print()
f = np.linalg.solve(A, b)
print("np.linalg.solve: ", f)

iteration  1 [3.1481481481481484, 3.5407407407407407, 1.9131687242798352]

iteration  2 [2.50303307422649, 3.5436976070720925, 1.9250605429389152]

iteration  3 [2.503256868275751, 3.5420225136311774, 1.9250874188535751]

iteration  4 [2.503631102071114, 3.5418692366577447, 1.9250833270605767]

iteration  5 [2.5036648605249883, 3.5418562788485946, 1.925082941863452]

np.linalg.solve:  [2.50366797 3.54185509 1.92508291]
